In [1]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import gsw
from glob import glob

In [2]:
temp_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/temperature/netcdf/decav/1.00/woa13_decav_t00_01.nc'
sal_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/salinity/netcdf/decav/1.00/woa13_decav_s00_01.nc'

In [3]:
temp = xr.open_dataset(temp_nc,decode_times=False)
sal = xr.open_dataset(sal_nc,decode_times=False)

In [4]:
# position of ADCP sent by Crissy
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [5]:
f = gsw.f(mlat)

In [6]:
ii = np.array(np.argmin(np.abs(mlat-temp['lat'])))
jj = np.array(np.argmin(np.abs(mlon-temp['lon'])))

In [7]:
mtemp_an = np.squeeze(temp['t_an'][0,:,ii,jj])
msal_an = np.squeeze(sal['s_an'][0,:,ii,jj])
mtemp_mn = np.squeeze(temp['t_mn'][0,:,ii,jj])
msal_mn = np.squeeze(sal['s_mn'][0,:,ii,jj])
mz = np.array(temp['depth'])

In [8]:
mpr = gsw.p_from_z(-mz,temp['lat'][ii])
mSA_mn = msal_mn
#mSA_mn = gsw.SA_from_SP(msal_mn, mpr, temp['lon'][jj], temp['lat'][ii])
mCT_mn = gsw.CT_from_t(mSA_mn,mtemp_mn,mpr)
msigma0 = gsw.sigma0(mSA_mn,mCT_mn)
msigma4 = gsw.sigma4(mSA_mn,mCT_mn)

mNsq,mpr_mid = gsw.Nsquared(mSA_mn,mCT_mn,mpr,lat=temp['lat'][ii])
mN = np.sqrt(mNsq)

In [9]:
ctd_dir = 'data/CTD_data_Station_M/'

file_list = glob(ctd_dir+'*CtdDer.txt')

#filename = '2006347tibrCtdDer.txt'

In [10]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [11]:
dz = 1
salspike_thresh = 0.05

In [12]:
dflist = []

plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

cnt = 0
for i,filename in enumerate(file_list):
    try:
        df = pd.read_csv(filename)
        
        
        #spi, = np.where((np.abs(np.diff(df['salin']))>0.2) | (np.abs(np.diff(df['pres']))>10))
        spi, = np.where((np.abs(np.diff(df['salin'][:-1]))>salspike_thresh) & 
                        (np.abs(np.diff(df['salin'][1:]))>salspike_thresh))
        spi = spi+1
        
        df['salin'][spi] = np.nan
        df['sigmat'][spi] = np.nan
        df['sigmatheta'][spi] = np.nan
        
        df['SA'] = gsw.SA_from_SP(df['salin'], df['pres'], mlon, mlat)
        df['CT'] = gsw.CT_from_t(df['SA'],df['temp'],df['pres'])
        df['sigma0'] = gsw.sigma0(df['SA'],df['CT'])
        df['sigma4'] = gsw.sigma4(df['SA'],df['CT'])
        
        df['zp'] = np.floor(df['pres']/dz+0.5)*dz
        dfm = df.groupby('zp').mean()
        
        #dfm['SA'] = dfm['SA'].rolling(25).median()
        #dfm['CT'] = dfm['CT'].rolling(25).median()
        dfm['SA'] = dfm['SA'].rolling(50,win_type='hanning').mean()
        dfm['CT'] = dfm['CT'].rolling(50,win_type='hanning').mean()
        #dfm['SA'] = dfm['sigmatheta'].rolling(25,win_type='hanning').mean()
        
        Nsq,pr_mid = gsw.Nsquared(dfm['SA'],dfm['CT'],dfm.index,lat=mlat)
        N = np.sqrt(Nsq)
        
        if i==0:
            dfall = dfm
        else:
            dfall = dfall.append(dfm)
        
        hrov, = ax1.plot(dfm['sigma4'],dfm.index,color='gray',lw=0.5)
        ax2.semilogx(N,pr_mid,color='gray',lw=0.5)
        
        cnt = cnt+1
        print(np.array(dfm['year'])[0],np.array(dfm['day'])[0])
    except:
        pass

hwod, = ax1.plot(msigma4,mpr,'k-',lw=2)
ax1.invert_yaxis()
ax1.legend((hrov,hwod),('ROV','World Ocean Atlas'))
ax1.set_ylabel('pressure [dbar]')
ax1.set_xlabel('$\sigma_4$ [kg/m$^3$]')
ax1.set_title('potential density\n4000 dbar reference')
xl = ax1.get_xlim()
yl = ax1.get_ylim()
ax1.set_ylim([yl[0],0])
ax1.text(xl[0]+0.04*np.diff(xl),yl[1]-0.1*np.diff(yl),'a)')

ax2.semilogx(mN,mpr_mid,'k-',lw=2)
ax2.invert_yaxis()
ax2.get_yaxis().set_ticklabels([])
ax2.set_title('buoyancy frequency')
ax2.set_xlabel('$N$ [rad s$^{-1}$]')
xl = ax2.get_xlim()
yl = ax2.get_ylim()
ax2.set_ylim([yl[0],0])
ax2.text(xl[0]+0.001*np.diff(xl),yl[1]-0.1*np.diff(yl),'b)')
plt.tight_layout()
plt.savefig('figures_paper/water_column_strat.pdf')
plt.savefig('figures_paper/water_column_strat.png')

FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2007.0 262.0
2007.0 157.0


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2007.0 266.0
2006.0 348.0
2007.0 261.0
2007.0 265.0
2007.0 33.0
2007.0 160.0
2007.0 159.0


/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in sqrt


2007.0 34.0
2007.0 35.0
2007.0 158.0
2007.0 31.0
2007.0 161.0
2007.0 32.0
2007.0 162.0
2006.0 347.0
2006.0 349.0
2007.0 264.0
2007.0 267.0
2007.0 156.0
2007.0 263.0


In [13]:
yl

(4262.349084084949, -200.33076578392317)

In [14]:
0.0007*86400

60.48

In [15]:
np.array(dfm['year'])[0]

2007.0

In [41]:
plt.figure()

for i,filename in enumerate(file_list):
    try:
        df = pd.read_csv(filename)
        
        
        #spi, = np.where((np.abs(np.diff(df['salin']))>0.2) | (np.abs(np.diff(df['pres']))>10))
        spi, = np.where((np.abs(np.diff(df['salin'][:-1]))>salspike_thresh) & 
                        (np.abs(np.diff(df['salin'][1:]))>salspike_thresh))
        spi = spi+1
        
        df['salin'][spi] = np.nan
        df['sigmat'][spi] = np.nan
        df['sigmatheta'][spi] = np.nan
        
        df['SA'] = gsw.SA_from_SP(df['salin'], df['pres'], mlon, mlat)
        df['CT'] = gsw.CT_from_t(df['SA'],df['temp'],df['pres'])
        df['sigma0'] = gsw.sigma0(df['SA'],df['CT'])
        df['sigma4'] = gsw.sigma4(df['SA'],df['CT'])
        
        df['zp'] = np.floor(df['pres']/dz+0.5)*dz
        dfm = df.groupby('zp').mean()
        
        #dfm['SA'] = dfm['SA'].rolling(25).median()
        #dfm['CT'] = dfm['CT'].rolling(25).median()
#         dfm['SA'] = dfm['SA'].rolling(50,win_type='hanning').mean()
#         dfm['CT'] = dfm['CT'].rolling(50,win_type='hanning').mean()
        #dfm['SA'] = dfm['sigmatheta'].rolling(25,win_type='hanning').mean()
        
        Nsq,pr_mid = gsw.Nsquared(dfm['SA'],dfm['CT'],dfm.index,lat=mlat)
        N = np.sqrt(Nsq)
        
        if i==0:
            dfall = dfm
        else:
            dfall = dfall.append(dfm)
        
        hrov, = plt.plot(df['CT'],df['pres'],color='gray',lw=0.5)
        plt.ylim([4025,3850])
        
        cnt = cnt+1
        print(np.array(dfm['year'])[0],np.array(dfm['day'])[0])
    except:
        pass
    
plt.xlim([1.1,1.3])

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tomconnolly/programs/anaconda/envs/py37/lib/pyth

2007.0 262.0
2007.0 157.0
2007.0 266.0
2006.0 348.0
2007.0 261.0
2007.0 265.0
2007.0 33.0
2007.0 160.0
2007.0 159.0
2007.0 34.0
2007.0 35.0
2007.0 158.0
2007.0 31.0
2007.0 161.0
2007.0 32.0
2007.0 162.0
2006.0 347.0
2006.0 349.0
2007.0 264.0
2007.0 267.0
2007.0 156.0
2007.0 263.0


(1.1, 1.3)